# Data Retraction from X

In [9]:
!pip install tweepy


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tweepy
import datetime
import pandas as pd

# Twitter API credentials (replace with your own)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAFnGzAEAAAAAw3J6M6RsI%2FG%2FR%2FnyoKemG9ezJlI%3D3VBkk61leyvfliYTLgD4HNghRLYtmMrNfn1upFcmykDisSxVlg"

# Authenticate with Twitter API
client = tweepy.Client(bearer_token=BEARER_TOKEN)


In [4]:
import tweepy
import datetime
import pandas as pd
import time

def wait_for_rate_limit():
    """Checks and waits for rate limit reset if necessary."""
    response = client.get_rate_limit_status()
    remaining_requests = response["resources"]["search"]["/search/tweets"]["remaining"]
    reset_time = response["resources"]["search"]["/search/tweets"]["reset"]
    
    if remaining_requests == 0:
        wait_time = max(reset_time - time.time(), 60)  # Ensure at least 60 seconds wait
        print(f"Rate limit reached. Waiting for {wait_time} seconds...")
        time.sleep(wait_time)



In [6]:
def fetch_tweets(keyword, start_date, end_date, max_results=100):
    """Fetch tweets containing a specific keyword within a date range with rate limit handling."""
    query = f"{keyword} -is:retweet lang:en"  # Filtering out retweets & non-English tweets
    tweets = []
    print(f"Starting tweet fetching for keyword: {keyword}")
    
    try:
        for tweet in tweepy.Paginator(client.search_recent_tweets, 
                                      query=query, 
                                      start_time=start_date, 
                                      end_time=end_date, 
                                      max_results=50,  # Reduce batch size to avoid hitting rate limits
                                      tweet_fields=["created_at", "text", "author_id"]):
            wait_for_rate_limit()  # Check rate limits before each request
            tweets.extend(tweet.data or [])
            print(f"Fetched {len(tweets)} tweets so far...")
            if len(tweets) >= max_results:
                break
            
            time.sleep(5)  # Small delay to avoid aggressive querying
    
    except tweepy.TooManyRequests as e:
        print("Rate limit reached. Waiting before retrying...")
        reset_time = int(e.response.headers.get("x-rate-limit-reset", time.time() + 60))  # Get reset time from headers
        wait_time = max(reset_time - time.time(), 60)  # Ensure at least 60 seconds wait
        time.sleep(wait_time)
        return fetch_tweets(keyword, start_date, end_date, max_results)  # Retry after waiting
    
    print("Tweet fetching completed.")
    return tweets[:max_results]

In [12]:
if __name__ == "__main__":
    keyword = "medical school" or "rama"
    tomorrow = (datetime.datetime.utcnow() + datetime.timedelta(days=1)).strftime("%Y-%m-%dT00:00:00Z")
    tomorrow_after = (datetime.datetime.utcnow() + datetime.timedelta(days=2)).strftime("%Y-%m-%dT00:00:00Z")
    start_date = tomorrow_after  # Start date from tomorrow
    end_date = tomorrow  # End date today
    max_results = 200
    
    print("Starting the script...")
    tweets = fetch_tweets(keyword, start_date, end_date, max_results)
    
    # Convert to DataFrame
    df = pd.DataFrame([{ "created_at": t.created_at, "text": t.text, "author_id": t.author_id } for t in tweets])
    print("Fetched tweets preview:")
    print(df.head())
    
    # Save to CSV
    df.to_csv("filtered_tweets.csv", index=False)
    print("Tweets saved to filtered_tweets.csv")

C:\Users\DIamond\AppData\Local\Temp\ipykernel_23700\3997376793.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  tomorrow = (datetime.datetime.utcnow() + datetime.timedelta(days=1)).strftime("%Y-%m-%dT00:00:00Z")
C:\Users\DIamond\AppData\Local\Temp\ipykernel_23700\3997376793.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  tomorrow_after = (datetime.datetime.utcnow() + datetime.timedelta(days=2)).strftime("%Y-%m-%dT00:00:00Z")


Starting the script...
Starting tweet fetching for keyword: medical school
Rate limit reached. Waiting before retrying...
Starting tweet fetching for keyword: medical school


BadRequest: 400 Bad Request
Invalid 'start_time':'2025-02-15T00:00Z'. 'start_time' must be a minimum of 10 seconds prior to the request time.
Invalid 'end_time':'2025-02-14T00:00Z'. 'end_time' must be a minimum of 10 seconds prior to the request time.
Invalid 'start_time':'2025-02-15T00:00Z'. 'start_time' must be before 'end_time':2025-02-14T00:00Z